In [ ]:
!pip install -U tensorboard-plugin-profile
!pip install wget
!pip install awscli
import os
os._exit(00)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=28d475c9016c02740e27cdcda674c83a0f3709a8f23d17b9171e96b90b3f428b
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 15.3 MB/s 
     |████████████████████████████████| 9.1 MB 46.2 MB/s 
     |████████████████████████████████| 548 kB 71.2 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 636 kB 85.0 MB/s 
     |████████████████████████████████| 140 kB 73.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall:

In [ ]:
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/MyDrive/config/awscli.ini
path="/content/drive/MyDrive/config/awscli.ini"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import wget
import numpy as np
import json
import os
import pickle
import yaml
import shutil
from tqdm import tqdm

os.environ['AWS_SHARED_CREDENTIALS_FILE']=path
import awscli

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

import warnings; warnings.simplefilter('ignore')

from drive.MyDrive.mushroom_classification.new import inat_src

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
CFG = yaml.safe_load(open('drive/MyDrive/mushroom_classification/new/src/config.YAML', 'rb'))

# AWS

In [ ]:
import tarfile
!aws s3 cp s3://ml-inat-competition-datasets/2021/train_mini.tar.gz .
!aws s3 cp s3://ml-inat-competition-datasets/2021/val.tar.gz .
!aws s3 cp s3://ml-inat-competition-datasets/2021/val.json.tar.gz .
!aws s3 cp s3://ml-inat-competition-datasets/2021/train_mini.json.tar.gz .

with tarfile.open('train_mini.tar.gz') as f:
    f.extractall()
with tarfile.open('val.tar.gz') as f:
    f.extractall()
with tarfile.open('val.json.tar.gz') as f:
    f.extractall()
with tarfile.open('train_mini.json.tar.gz') as f:
    f.extractall()

download: s3://ml-inat-competition-datasets/2021/train_mini.tar.gz to ./train_mini.tar.gz
download: s3://ml-inat-competition-datasets/2021/val.tar.gz to ./val.tar.gz
download: s3://ml-inat-competition-datasets/2021/val.json.tar.gz to ./val.json.tar.gz
download: s3://ml-inat-competition-datasets/2021/train_mini.json.tar.gz to ./train_mini.json.tar.gz


In [ ]:
val = src.read_aws_jsons('val.json')
train_mini = src.read_aws_jsons('train_mini.json')

remove_images_not_fungi('val', val)
remove_images_not_fungi('train_mini', train_mini)

train_mini, val = create_labels(train_mini, val)
train_mini, val = create_filepaths(train_mini, val)

# Creating Datasets

In [ ]:
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow.keras import applications
AUTOTUNE = tf.data.AUTOTUNE

t_paths = train_mini['filepath']
v_paths = val['filepath']
t_class = train_mini['label']
v_class = val['label']

train_ds = tf.data.Dataset.from_tensor_slices((t_paths, t_class)).map(
    lambda filepath, label: (load_img(filepath), label), 
    num_parallel_calls=AUTOTUNE).\
    cache().\
    shuffle(20000).\
    batch(CFG['BATCH_SIZE']).\
    prefetch(buffer_size=AUTOTUNE)
    
val_ds = tf.data.Dataset.from_tensor_slices((v_paths, v_class)).map(
    lambda filepath, label: (load_img(filepath), label), 
    num_parallel_calls=AUTOTUNE).\
    cache().\
    shuffle(4000).\
    batch(CFG['BATCH_SIZE']).\
    prefetch(buffer_size=AUTOTUNE)

In [ ]:
%load_ext tensorboard
import datetime
from tensorflow.keras import applications 
from tensorflow.keras import layers
image_size = CFG['IMAGE_SIZE']
!rm -rf ./logs/
AUTOTUNE = tf.data.experimental.AUTOTUNE

def lr_scheduler(epoch):
    return CFG['INITIAL_LR_RATE'] * tf.math.pow(CFG['LR_DECAY_FACTOR'], epoch//CFG['EPOCHS_PER_DECAY'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
optimizer = tf.keras.optimizers.Nadam(CFG['INITIAL_LR_RATE'])

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=CFG['ES_PATIENCE']),

    tf.keras.callbacks.LearningRateScheduler(
        lr_scheduler, verbose=1),

    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch='550, 600')
]

def create_model():
    input = layers.Input(shape=image_size+[3], dtype=tf.float32)
    base = applications.efficientnet_v2.EfficientNetV2B0(include_top=False, input_tensor=input)
    avg = layers.GlobalAveragePooling2D()(base.output)
    dropout = layers.Dropout(CFG['DROPOUT_PERCENT'])(avg)
    x = layers.Dense(341, activation='softmax')(dropout)
    model = keras.Model(inputs=base.input, outputs=x)
    return model

In [ ]:
num_train_examples = train_mini.shape[0]
num_val_examples = val.shape[0]
STEPS_PER_EPOCH = np.ceil(num_train_examples/CFG["BATCH_SIZE"])
VAL_STEPS = np.ceil(num_val_examples/CFG["BATCH_SIZE"])
model = create_model()
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3 accuracy')])

24289280/24274472 [==============================] - 0s 0us/step


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
%time
history = model.fit(train_ds,
                    validation_data=val_ds,
                    validation_steps=VAL_STEPS,
                    epochs=5,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    callbacks=callbacks)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs

Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/5
267/267 [==============================] - 2904s 11s/step - loss: 5.1880 - accuracy: 0.0295 - top3 accuracy: 0.0067 - val_loss: 4.7603 - val_accuracy: 0.0440 - val_top3 accuracy: 0.0097 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/5
 48/267 [====>.........................] - ETA: 36:02 - loss: 4.3132 - accuracy: 0.0895 - top3 accuracy: 0.0150

KeyboardInterrupt: ignored